# Moduling

In [1]:
import pandas as pd
import numpy as np
import sklearn

In [2]:
URL  = "https://raw.githubusercontent.com/Diyorbek-MY/CA_Housing_Values/main/housing.csv"
df = pd.read_csv(URL)

In [3]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

X_train = train_set.drop("median_house_value", axis=1)
y = train_set["median_house_value"].copy()

X_num = X_train.drop("ocean_proximity", axis=1)

# building Pipeline

In [4]:
from sklearn.base import BaseEstimator, TransformerMixin
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler()),
])

In [6]:
from sklearn.compose import ColumnTransformer

num_attribs = list(X_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", OneHotEncoder(), cat_attribs),
])

In [7]:
X_prepared = full_pipeline.fit_transform(X_train)

In [8]:
X_prepared

array([[ 1.27258656, -1.3728112 ,  0.34849025, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.70916212, -0.87669601,  1.61811813, ...,  0.        ,
         0.        ,  1.        ],
       [-0.44760309, -0.46014647, -1.95271028, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.59946887, -0.75500738,  0.58654547, ...,  0.        ,
         0.        ,  0.        ],
       [-1.18553953,  0.90651045, -1.07984112, ...,  0.        ,
         0.        ,  0.        ],
       [-1.41489815,  0.99543676,  1.85617335, ...,  0.        ,
         1.        ,  0.        ]])

# Linear Regression

In [9]:

from sklearn.linear_model import LinearRegression

LR_model = LinearRegression()

In [10]:
LR_model.fit(X_prepared, y)

LinearRegression()

In [11]:
test_data = X_train.sample(10)
test_data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
2804,-119.03,35.45,14.0,3520.0,604.0,1748.0,582.0,4.3162,INLAND
2333,-119.69,36.83,8.0,943.0,189.0,475.0,155.0,4.9327,INLAND
18783,-122.33,40.47,30.0,2502.0,523.0,1296.0,481.0,2.1250,INLAND
14391,-117.24,32.74,45.0,1718.0,293.0,757.0,329.0,4.0500,NEAR OCEAN
18488,-121.59,36.97,16.0,865.0,123.0,403.0,130.0,5.7396,INLAND
1149,-121.56,39.52,9.0,818.0,197.0,358.0,197.0,1.7708,INLAND
506,-122.29,37.84,35.0,1872.0,419.0,1017.0,414.0,2.2106,NEAR BAY
5499,-118.42,34.00,14.0,3771.0,1180.0,2355.0,978.0,3.1603,<1H OCEAN
20413,-118.87,34.18,21.0,5661.0,1369.0,3188.0,1308.0,3.4676,<1H OCEAN
4350,-118.37,34.11,42.0,5518.0,979.0,1863.0,957.0,8.5842,<1H OCEAN


In [12]:
X_train

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
14196,-117.03,32.71,33.0,3126.0,627.0,2300.0,623.0,3.2596,NEAR OCEAN
8267,-118.16,33.77,49.0,3382.0,787.0,1314.0,756.0,3.8125,NEAR OCEAN
17445,-120.48,34.66,4.0,1897.0,331.0,915.0,336.0,4.1563,NEAR OCEAN
14265,-117.11,32.69,36.0,1421.0,367.0,1418.0,355.0,1.9425,NEAR OCEAN
2271,-119.80,36.78,43.0,2382.0,431.0,874.0,380.0,3.5542,INLAND
...,...,...,...,...,...,...,...,...,...
11284,-117.96,33.78,35.0,1330.0,201.0,658.0,217.0,6.3700,<1H OCEAN
11964,-117.43,34.02,33.0,3084.0,570.0,1753.0,449.0,3.0500,INLAND
5390,-118.38,34.03,36.0,2101.0,569.0,1756.0,527.0,2.9344,<1H OCEAN
860,-121.96,37.58,15.0,3575.0,597.0,1777.0,559.0,5.7192,<1H OCEAN


In [13]:
test_label = y.loc[test_data.index]
test_label

,median_house_value
2804,87100.0
2333,89500.0
18783,66100.0
14391,284900.0
18488,308700.0
1149,79500.0
506,132500.0
5499,287500.0
20413,212800.0
4350,500001.0


In [14]:
test_data_prepared = full_pipeline.transform(test_data)
predicted_labels = LR_model.predict(test_data_prepared)

In [15]:
predicted_labels

array([163579.96758493, 167352.3674967 ,  62308.35995356, 248677.65719237,
       243563.76169267,  32985.60916878, 172910.04820283, 230878.69435656,
       248655.82514456, 469545.56451302])

In [16]:
pd.DataFrame({'Predicted': predicted_labels, 'Actual': test_label})

,Predicted,Actual
2804,163579.967585,87100.0
2333,167352.367497,89500.0
18783,62308.359954,66100.0
14391,248677.657192,284900.0
18488,243563.761693,308700.0
1149,32985.609169,79500.0
506,172910.048203,132500.0
5499,230878.694357,287500.0
20413,248655.825145,212800.0
4350,469545.564513,500001.0


#model evaluation

In [17]:
test_set

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
20046,-119.01,36.06,25.0,1505.0,NaN,1392.0,359.0,1.6812,47700.0,INLAND
3024,-119.46,35.14,30.0,2943.0,NaN,1565.0,584.0,2.5313,45800.0,INLAND
15663,-122.44,37.80,52.0,3830.0,NaN,1310.0,963.0,3.4801,500001.0,NEAR BAY
20484,-118.72,34.28,17.0,3051.0,NaN,1705.0,495.0,5.7376,218600.0,<1H OCEAN
9814,-121.93,36.62,34.0,2351.0,NaN,1063.0,428.0,3.7250,278000.0,NEAR OCEAN
...,...,...,...,...,...,...,...,...,...,...
15362,-117.22,33.36,16.0,3165.0,482.0,1351.0,452.0,4.6050,263300.0,<1H OCEAN
16623,-120.83,35.36,28.0,4323.0,886.0,1650.0,705.0,2.7266,266800.0,NEAR OCEAN
18086,-122.05,37.31,25.0,4111.0,538.0,1585.0,568.0,9.2298,500001.0,<1H OCEAN
2144,-119.76,36.77,36.0,2507.0,466.0,1227.0,474.0,2.7850,72300.0,INLAND


In [18]:
x_test = test_set.drop("median_house_value", axis=1)
y_test = test_set["median_house_value"].copy()
y_test

,median_house_value
20046,47700.0
3024,45800.0
15663,500001.0
20484,218600.0
9814,278000.0
...,...
15362,263300.0
16623,266800.0
18086,500001.0
2144,72300.0


In [19]:
x_test_prepared = full_pipeline.transform(x_test)

In [20]:
y_predicted = LR_model.predict(x_test_prepared)

In [21]:
# MAE and RMSE

In [22]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_test, y_predicted)
mae

50898.73953494079

In [23]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_predicted)
rmse = np.sqrt(mse)
rmse

np.float64(72701.32600762135)

## Random Forest

In [24]:
from sklearn.ensemble import RandomForestRegressor
RF_model = RandomForestRegressor()
RF_model.fit(X_prepared, y)

RandomForestRegressor()

In [27]:
y_predicted = RF_model.predict(x_test_prepared)

In [28]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_predicted)
print("RMSE=", np.sqrt(mse))

RMSE= 50247.638749387006


## Cross-validation

In [29]:
x = df.drop("median_house_value", axis=1)
y = df["median_house_value"].copy()

x_prepared = full_pipeline.fit_transform(x)

In [33]:
from sklearn.model_selection import cross_val_score

mse_scores = cross_val_score(LR_model, x_prepared, y, scoring="neg_mean_squared_error", cv=5)

In [34]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

In [35]:
display_scores(np.sqrt(-mse_scores))

Scores: [73391.42036892 74809.28332317 75429.91837496 76604.35506436
 66196.72436926]
Mean: 73286.34030013601
Standard deviation: 3693.161254481324


In [36]:
scores = cross_val_score(RF_model, x_prepared, y, scoring="neg_mean_squared_error", cv=10)
LR_rmse_scores = np.sqrt(-scores)
display_scores(LR_rmse_scores)

Scores: [98570.31094764 47454.58116323 65413.34541283 56424.94448222
 61045.08994204 59693.62495665 47466.38844363 78839.71210792
 74444.5907304  49391.27360845]
Mean: 63874.386179501504
Standard deviation: 15399.469454663324


## presentation

pickle

In [38]:
import pickle

filename = 'RF_model.pkl'
with open(filename, 'wb') as file:
    pickle.dump(RF_model, file)

In [39]:
with open(filename, 'rb') as file:
    RF_model = pickle.load(file)

joblib

In [43]:
import joblib

filename = 'LR_model.jbl'
joblib.dump(LR_model, filename)

['LR_model.jbl']

In [44]:
model = joblib.load(filename)

In [46]:
scores = cross_val_score(model, x_prepared, y, scoring="neg_mean_squared_error", cv=5)
LR_rmse_scores = np.sqrt(-scores)
display_scores(LR_rmse_scores)

Scores: [73391.42036892 74809.28332317 75429.91837496 76604.35506436
 66196.72436926]
Mean: 73286.34030013601
Standard deviation: 3693.161254481324
